![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/SOCIAL_DETERMINANT_CLASSIFICATION.ipynb)

# **Social Determinants of Health-Classification**

📌To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.

# **Colab Setup**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [ ]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.1.0
Spark NLP_JSL Version : 5.1.0


# 🔎 MODELS

### Sequence Classifier :
|                       Model Name                       |                                                        Description                                                       |   Language  |
|:------------------------------------------------------:|:------------------------------------------------------------------------------------------------------------------------:|:-----------:|
| bert_sequence_classifier_sdoh_community_present_status | This model classifies related to social support such as a family member or friend in the clinical documents.             | True, False |
| bert_sequence_classifier_sdoh_community_absent_status  | This model classifies related to the loss of social support such as a family member or friend in the clinical documents. | True, False |

### Generic Classifier :
|                              Model Name                             |                                                                    Description                                                                    |                              Language                              |
|:-------------------------------------------------------------------:|:-------------------------------------------------------------------------------------------------------------------------------------------------:|:------------------------------------------------------------------:|
| genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli             | This Generic Classifier model is intended for detecting alcohol use in clinical notes and trained by using GenericClassifierApproach annotator.   | Present, Past, Never, None                                         |
| genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli             | This Generic Classifier model is intended for detecting tobacco use in clinical notes and trained by using GenericClassifierApproach annotator.   | Present, Past, Never, None                                         |
| genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli    | This Generic Classifier model is intended for detecting substance use in clinical notes and trained by using GenericClassifierApproach annotator. | Present, None                                                      |
| genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli      | This Generic Classifier model is intended for detecting alcohol use in clinical notes and trained by using GenericClassifierApproach annotator.   | Present, Never, None                                               |
| genericclassifier_sdoh_economics_binary_sbiobert_cased_mli          | This model classifies related to social economics status in the clinical documents and trained by using GenericClassifierApproach annotator.      | True, False                                                        |
| genericclassifier_sdoh_transportation_insecurity_sbiobert_cased_mli | The transportation insecurity classifier employs BERT embeddings within a generic classifier framework.                                           | Transportation_Insecurity, No_Transportation_Insecurity_Or_Unknown |





**🔎You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?task=Named+Entity+Recognition&edition=Spark+NLP+for+Healthcare)**

# 📌 Sequence Classifier

### **`bert_sequence_classifier_sdoh_community_present_status`**

In [ ]:
text_list = [
" Patient with history of lupus, lupus nephritis with ESRD on peritoneal dialysis on transplant list, hx of PE/Antiphopholipid antibody on coumadin, mitral regurg, presents with 4-6 month history of cough, worse in the morning, one week of trace blood, now producing bright red blood over last couple days. Patient states that the amount of blood she has been coughing has been increasing and is now almost hourly, aprroximately 1 teaspoon bright red blood. Patient states that the cough produced primarily yellow sputum until it turned to blood. Patient denies any other symptoms such as dizziness or lightheadedness.  Married with three children,Worked as an accountant until health declined in early 2002. No tobacco, ethanol or drug use. Centrilobular nodules and ground glass opacities throughout both lungs, with a basilar predominance, with associated mild bronchiectasis, compatible with chronic collagen vascular disease, progressed since 2002. There is no advanced fibrosis. Superimposed infection cannot be excluded by imaging alone. Ground glass opacities could also represent hemorrhage. 3. Chronic left lower segmental pulmonary arterial PE, unchanged since 2191. No new acute PE detected to the subsegmental levels.",
" This is an 87 year old man status post motor vehicle accident in Month (only) 956 who was recently discharged from Hospital1 18 status post left radical nephrectomy for renal oncocytoma, who returned to Hospital1 18 on 3-22 for outpatient followup CT scan of the head. Patient was found to have a left subdural hematoma and was transported to the emergency department for workup. Currently patient does not complain of fever, chills, nausea, vomiting, chest pain, shortness of breath. No known drug allergies.The patient is a retired priest. Denies history of tobacco or alcohol use. Patient currently lives at Hospital3 2558.",
]

In [ ]:
documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_sdoh_community_present_status", "en", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")


pipeline = Pipeline(stages=[
                        documentAssembler,
                        tokenizer,
                        sequenceClassifier])


df = spark.createDataFrame(text_list, StringType()).toDF("text")
results = pipeline.fit(df).transform(df)

bert_sequence_classifier_sdoh_community_present_status download started this may take some time.
[OK!]


In [ ]:
res = results.select(F.explode(F.arrays_zip(results.document.result,
                                             results.class_.result,
                                             results.class_.metadata)).alias("col"))\
               .select(F.expr("col['1']").alias("prediction"),
                       F.expr("col['2']").alias("confidence"),
                       F.expr("col['0']").alias("sentence"))

if res.count()>1:
  udf_func = F.udf(lambda x,y:  x["Some("+str(y)+")"])
  res.withColumn('confidence', udf_func(res.confidence, res.prediction)).show(truncate=150)

+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|confidence|                                                                                                                                              sentence|
+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|      True| 0.9982032| Patient with history of lupus, lupus nephritis with ESRD on peritoneal dialysis on transplant list, hx of PE/Antiphopholipid antibody on coumadin,...|
|     False|0.78874034| This is an 87 year old man status post motor vehicle accident in Month (only) 956 who was recently discharged from Hospital1 18 status post left r...|
+----------+----------+------------------------------------------------------------------------------------------------------

### **`bert_sequence_classifier_sdoh_community_absent_status`**

In [ ]:
text_list = [
"She has two adult sons. She is a widow. She was employed with housework. She quit smoking 20 to 30 years ago, but smoked two packs per day for 20 to 30 years. She drinks one glass of wine occasionally. She avoids salt in her diet. ",
"65 year old male presented with several days of vice like chest pain. He states that he felt like his chest was being crushed from back to the front. Lives with spouse and two sons moved to US 1 month ago.",
]

In [ ]:
sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_sdoh_community_absent_status", "en", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")


pipeline = Pipeline(stages=[
                      documentAssembler,
                      tokenizer,
                      sequenceClassifier])


df = spark.createDataFrame(text_list, StringType()).toDF("text")
results = pipeline.fit(df).transform(df)

bert_sequence_classifier_sdoh_community_absent_status download started this may take some time.
[OK!]


In [ ]:
res = results.select(F.explode(F.arrays_zip(results.document.result,
                                             results.class_.result,
                                             results.class_.metadata)).alias("col"))\
               .select(F.expr("col['1']").alias("prediction"),
                       F.expr("col['2']").alias("confidence"),
                       F.expr("col['0']").alias("sentence"))

if res.count()>1:
  udf_func = F.udf(lambda x,y:  x["Some("+str(y)+")"])
  res.withColumn('confidence', udf_func(res.confidence, res.prediction)).show(truncate=150)

+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|confidence|                                                                                                                                              sentence|
+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|      True| 0.9894813|She has two adult sons. She is a widow. She was employed with housework. She quit smoking 20 to 30 years ago, but smoked two packs per day for 20 t...|
|     False|0.72528476|65 year old male presented with several days of vice like chest pain. He states that he felt like his chest was being crushed from back to the fron...|
+----------+----------+------------------------------------------------------------------------------------------------------

# 📌 Generic Classifier

### **`genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli`**

In [ ]:
text_list = ["Retired schoolteacher, now substitutes. Lives with wife in location 1439. Has a 27 yo son and a 25 yo daughter. He uses alcohol and cigarettes",
             "The patient quit smoking approximately two years ago with an approximately a 40 pack year history, mostly cigar use. He also reports 'heavy alcohol use', quit 15 months ago.",
             "The patient denies any history of smoking or alcohol abuse. She lives with her one daughter.",
             "She was previously employed as a hairdresser, though says she hasnt worked in 4 years. Not reported by patient, but there is apparently a history of alochol abuse."
             ]

df = spark.createDataFrame(text_list, StringType()).toDF("text")

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

generic_classifier = GenericClassifierModel.pretrained("genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli", 'en', 'clinical/models')\
    .setInputCols(["features"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_embeddings,
    features_asm,
    generic_classifier
])

results = pipeline.fit(df).transform(df)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli download started this may take some time.
[OK!]


In [ ]:
res = results.select(F.explode(F.arrays_zip(results.document.result,
                                             results.class_.result,
                                             results.class_.metadata)).alias("col"))\
               .select(F.expr("col['1']").alias("prediction"),
                       F.expr("col['2']['confidence']").alias("confidence"),
                       F.expr("col['0']").alias("sentence"))

res.show(truncate=150)

+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|confidence|                                                                                                                                              sentence|
+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|   Present|0.65745443|        Retired schoolteacher, now substitutes. Lives with wife in location 1439. Has a 27 yo son and a 25 yo daughter. He uses alcohol and cigarettes|
|      Past|0.98161787|The patient quit smoking approximately two years ago with an approximately a 40 pack year history, mostly cigar use. He also reports 'heavy alcohol...|
|     Never| 0.9825732|                                                          The patient denies any history of smoking or

### **`genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli`**

In [ ]:
text_list = ["Retired schoolteacher, now substitutes. Lives with wife in location 1439. Has a 27 yo son and a 25 yo daughter. He uses alcohol and cigarettes",
             "The patient quit smoking approximately two years ago with an approximately a 40 pack year history, mostly cigar use. He also reports 'heavy alcohol use', quit 15 months ago.",
             "Employee in neuro departmentin at the Center Hospital 18. Widower since 2001. Current smoker since 20 years. No EtOH or illicits.",
             "Patient smoked 4 ppd x 37 years, quitting 22 years ago. He is widowed, lives alone, has three children."
             ]

df = spark.createDataFrame(text_list, StringType()).toDF("text")

In [ ]:
generic_classifier = GenericClassifierModel.pretrained("genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli", 'en', 'clinical/models')\
    .setInputCols(["features"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_embeddings,
    features_asm,
    generic_classifier
])

results = pipeline.fit(df).transform(df)


In [ ]:
res = results.select(F.explode(F.arrays_zip(results.document.result,
                                             results.class_.result,
                                             results.class_.metadata)).alias("col"))\
               .select(F.expr("col['1']").alias("prediction"),
                       F.expr("col['2']['confidence']").alias("confidence"),
                       F.expr("col['0']").alias("sentence"))

res.show(truncate=150)

+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|confidence|                                                                                                                                              sentence|
+----------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|   Present|0.65745443|        Retired schoolteacher, now substitutes. Lives with wife in location 1439. Has a 27 yo son and a 25 yo daughter. He uses alcohol and cigarettes|
|      Past|0.98161787|The patient quit smoking approximately two years ago with an approximately a 40 pack year history, mostly cigar use. He also reports 'heavy alcohol...|
|     Never| 0.9825732|                                                          The patient denies any history of smoking or

### **`genericclassifier_sdoh_transportation_insecurity_sbiobert_cased_mli`**

In [ ]:
text_list =[
"Patient B is a 40-year-old female who was diagnosed with breast cancer. She has received a treatment plan that includes surgery, chemotherapy, and radiation therapy. She is alone and can not drive a car or can not use public bus.",
"Emily, a 30-year-old woman, had been struggling to manage her chronic condition due to transportation problems. She often missed medical appointments and couldn't obtain her prescribed medications regularly. Recognizing the impact of transportation barriers on her health, her healthcare provider worked to find solutions. They helped her arrange rides through community programs and provided her with information about telehealth options. By addressing her transportation challenges, Emily's healthcare provider ensured that she could better manage her health and receive the necessary care.",
"""Patient John is a 60-year-old man who presents to a primary care clinic for a routine check-up. He reports feeling generally healthy, with no significant medical concerns. However, he reveals that he is a smoker and drinks alcohol on a regular basis. The patient also mentions that he has a history of working long hours and has limited time for physical activity and social interactions.



Based on this information, it appears that Patient John's overall health may be affected by several social determinants of health, including tobacco and alcohol use, lack of physical activity, and social isolation. To address these issues, the healthcare provider may recommend a comprehensive physical exam and develop a treatment plan that includes lifestyle modifications, such as smoking cessation and reduction of alcohol intake. Additionally, the patient may benefit from referrals to local organizations that provide resources for physical activity and social engagement. The healthcare provider may also recommend strategies to reduce work-related stress and promote work-life balance. By addressing these social determinants of health, healthcare providers can help promote Patient John's overall health and prevent future health problems.

Additionally, The distance to the hospital from the John's home is considerable, and there are no available train services.""",
"Ms. Klnum is a 69y/o lady with COPD, ulcerative colitis, diverticulosis s/p partial colectomy, and 2 PE's in the past on chronic Warfarin with IVC filter who presented to the ED for chest pain and was found to have PEs and pneumonia. Cough productive of yellow sputum and has been ongoing for weeks, no acute change.  She is divorced. Her 2 daughters died of drug use. She is close with her sister who is her HCP, as well as a brother. -Tobacco: Smoked ~2ppd from age 13 until 5 years ago. -EtOH: former heavy use, reports drinking two 6 packs per day for 2 yrs; quit 27 yrs ago. -Illicits: None. Family History: Daughter - colitis. Had 6 siblings. One sister died, 35, ovarian CA. Brother, died at 48, stroke.",
"Mark, a 35-year-old man, sought medical help for symptoms of attention-deficit/hyperactivity disorder (ADHD), a neurodevelopmental disorder characterized by inattention, hyperactivity, and impulsivity. After a comprehensive evaluation, Mark was diagnosed with ADHD, and his healthcare provider recommended a multimodal treatment approach. ",
"Michael, a 25-year-old man, sought medical advice for concerns related to stress and difficulty managing everyday pressures. After a thorough evaluation, it was determined that Michael did not meet the criteria for any specific mental disorder. However, his healthcare provider acknowledged the impact of stress on his well-being and recommended implementing stress management techniques. They discussed strategies such as regular exercise, relaxation techniques, and engaging in activities that bring him joy and relaxation. "
"John, a 45-year-old man, was diagnosed with bipolar disorder, a mental disorder characterized by alternating periods of elevated mood (mania) and depression. His treatment plan involved a combination of mood stabilizing medication and regular therapy sessions. With proper management and support, John learned to better understand and cope with his condition, leading to improved stability and overall well-being.",
],

df = spark.createDataFrame(text_list, StringType()).toDF("text")

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

generic_classifier = GenericClassifierModel.pretrained("genericclassifier_sdoh_transportation_insecurity_sbiobert_cased_mli", 'en', 'clinical/models')\
    .setInputCols(["features"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_embeddings,
    features_asm,
    generic_classifier
])

In [ ]:
results = pipeline.fit(df).transform(df)

In [ ]:
res = results.select(F.explode(F.arrays_zip(results.document.result,
                                             results.class_.result,
                                             results.class_.metadata)).alias("col"))\
               .select(F.expr("col['1']").alias("prediction"),
                       F.expr("col['2']['confidence']").alias("confidence"),
                       F.expr("col['0']").alias("sentence"))

res.show(truncate=150)

+-------------------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|               prediction|confidence|                                                                                                                                              sentence|
+-------------------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|Transportation_Insecurity| 0.9798329|[Patient B is a 40-year-old female who was diagnosed with breast cancer. She has received a treatment plan that includes surgery, chemotherapy, and...|
+-------------------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------+

